In [2]:
import gymnasium as gym

# Running the environment
In this example im going to be using [Lunar Lander](https://gymnasium.farama.org/environments/box2d/lunar_lander/). One of the presets for this game is `"LunarLander-V2"` so i will be using this when creating the environment.

This process isn't training the agent at all right now we are just selecting a random action with `action = env.action_space.sample()` and taking that action with `env.step(action)`

The `env.render()` allows use to see hte agent playing the game in it's own window  with `render_mode="human"`

This condition just checks to see if the environment has stopped and then reseting it with `env.reset()` when met
```py
if terminated or truncated:
    observation, info = env.reset()
```


In [3]:
env = gym.make("LunarLander-v2", render_mode="rgb_array")
observation, info = env.reset()

for _ in range(1000):
    env.render()
    action = env.action_space.sample()
    observation, reward, terminated, truncated, info = env.step(action)

    if terminated or truncated:
        observation, info = env.reset()

# Training

In [4]:
from stable_baselines3 import DQN

model = DQN("MlpPolicy", env, verbose=1)
model.learn(total_timesteps=10000, log_interval=4)
model.save("dqn_cartpole")

del model # remove to demonstrate saving and loading

model = DQN.load("dqn_cartpole")

obs, info = env.reset()
while True:
    action, _states = model.predict(obs, deterministic=True)
    obs, reward, terminated, truncated, info = env.step(action)
    if terminated or truncated:
        obs, info = env.reset()

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 90.8     |
|    ep_rew_mean      | -184     |
|    exploration_rate | 0.655    |
| time/               |          |
|    episodes         | 4        |
|    fps              | 2429     |
|    time_elapsed     | 0        |
|    total_timesteps  | 363      |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 1.18     |
|    n_updates        | 65       |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 126      |
|    ep_rew_mean      | -642     |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 8        |
|    fps              | 2047     |
|    time_elapsed     | 0        |
|    total_timesteps  | 1004     |
| train/              |        

KeyboardInterrupt: 